In [1]:
import pandas as pd
from yaml import load
from yaml import CLoader as Loader
from abil.tune import tune
from abil.functions import example_data
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'abil'

In [2]:
#If you want to debug the package without repackaging, you can read files locally:

import sys
sys.path.insert(0, '/home/phyto/Abil/abil/')
from tune import tune
from functions import example_data

In [4]:
# Setting up a regressor model

with open('/home/phyto/Abil/configuration/ensemble_regressor.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

predictors = model_config['predictors']
d = pd.read_csv("/home/phyto/Abil/devries2024/data/obs_env.csv")

species = 'Rhabdosphaera xiphos'

# turn NaNs into zeros:
d[species] = d[species].fillna(0)

d.dropna(subset=species, inplace=True)
d.dropna(subset='FID', inplace=True)
X_train = d[predictors]
y = d[species]
X_predict = pd.read_csv("/home/phyto/Abil/devries2024/data/envdata_final.csv").head(100)
X_predict.set_index(["time", "depth", "lat", "lon"], inplace=True)
X_predict = X_predict[X_train.columns]


m = tune(X_train, y, model_config, regions="FID")

upsampling = True


In [4]:
'''
2-phase XGboost regressor
'''
m.train(model="xgb", regressor=True, classifier=True)

training regressor
{'regressor__estimator__learning_rate': [0.05], 'regressor__estimator__n_estimators': [100], 'regressor__estimator__max_depth': [7], 'regressor__estimator__subsample': [0.8], 'regressor__estimator__colsample_bytree': [0.5], 'regressor__estimator__gamma': [1], 'regressor__estimator__reg_alpha': [0.1]}
Fitting 10 folds for each of 1 candidates, totalling 10 fits
exported model to: /home/phyto/planktonSDM/ModelOutput/xgb/model/Rhabdosphaera xiphos_reg.sav
exported scoring to: /home/phyto/planktonSDM/ModelOutput/xgb/scoring/Rhabdosphaera xiphos_reg.sav
reg rRMSE: 167%
reg rMAE: 81%
reg R2: -0.08
execution time: 7.152557373046875e-07 seconds


In [5]:
'''
2-phase Random forest regressor
'''
m.train(model="rf", regressor=True, classifier=True)

training regressor
{'regressor__estimator__n_estimators': [100], 'regressor__estimator__max_features': [4], 'regressor__estimator__max_depth': [50], 'regressor__estimator__min_samples_leaf': [0.5, 1], 'regressor__estimator__max_samples': [0.5, 1]}
Fitting 10 folds for each of 4 candidates, totalling 40 fits


KeyboardInterrupt: 

In [ ]:
'''
1-phase XGboost regressor
'''
m.train(model="xgb", regressor=True, classifier=True)

training regressor
Fitting 10 folds for each of 72 candidates, totalling 720 fits
exported model to: /home/phyto/CoccoML/ModelOutput/xgb/model/Emiliania huxleyi_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/xgb/scoring/Emiliania huxleyi_reg.sav
reg rRMSE: 358%
reg rMAE: 86%
reg R2: 0.14
execution time: 7.152557373046875e-07 seconds


In [ ]:
'''
1-phase KNN regressor
'''
m.train(model="knn", regressor=True)

training regressor
Fitting 10 folds for each of 1 candidates, totalling 10 fits


exported model to: /home/phyto/CoccoML/ModelOutput/knn/model/Emiliania huxleyi_reg.sav
exported scoring to: /home/phyto/CoccoML/ModelOutput/knn/scoring/Emiliania huxleyi_reg.sav
reg rRMSE: 367%
reg rMAE: 97%
reg R2: 0.0
execution time: 9.5367431640625e-07 seconds
